In [1]:
!pip install rdflib

In [2]:
import re
import rdflib

Knowledge

In [3]:
URI_PREFIX = 'http://kgdemo.com/'

triples = [
  ('宝马', 'is_what', '宝马（BMW）是德国豪华汽车品牌'),
  ('宝马', 'is_how', '德系大品牌值得信赖，各方面口碑都很好'),
  ('宝马', 'is_compared', '各有千秋，但是人生苦短，我选宝马'),
  ('捷豹', 'is_what', '捷豹（Jaguar）英国豪华汽车品牌，英国皇室御用品牌，1935年诞生'),
  ('宾利', 'is_how', '举世闻名的豪华汽车制造品牌，非常昂贵哦'),
  ('帕加尼', 'is_what', '帕加尼（Pagani）是一家位于意大利摩德纳的超级跑车制造商，该车厂坚持手工打造车辆，其汽车产量非常少，价格也十分昂贵'),
  ('广汽本田', 'is_what', '广汽本田汽车有限公司（原广州本田汽车有限公司；简称广汽本田）于1998年7月1日成立，它是由广州汽车集团公司与日本本田技研工业株式会社共同出资组建的合资公司，双方各占50%股份，合作年限为30年'),
  ('北京奔驰', 'is_how', '大品牌值得信赖，我经常在宝马的后视镜里看到它'),
]

graph = rdflib.Graph()
resources = set([r for triple in triples for r in triple])
resource2uri = {r: URI_PREFIX + r for r in resources}
uri2resource = {uri: r for r, uri in resource2uri.items()}
for (s, p, o) in triples:
  s_uri = rdflib.URIRef(resource2uri[s])
  p_uri = rdflib.URIRef(resource2uri[p])
  o_uri = rdflib.URIRef(resource2uri[o])
  graph.add((s_uri, p_uri, o_uri))

Rule

In [4]:
class Intent(object):
  CarIsWhat = 'car_is_what'
  CarIsHow = 'car_is_how'
  CarIsCompared = 'car_is_compared'

In [5]:
rules = [
  (Intent.CarIsWhat, r"(?:我想|给我)(?:了解|介绍|讲解)(?:一下)?(.*)(?:这个汽车品牌)"),
  (Intent.CarIsWhat, r"(?:我想|给我)(?:了解|介绍|讲解)(?:一下)?(.*)$"),
  (Intent.CarIsCompared, r"(.*)和.*比(?:怎么样|哪个好|哪个更好)"),
  (Intent.CarIsHow, r"(.*?)(?:这个牌子的汽车|这个汽车品牌|汽车)?(?:怎么样|好用吗|如何呢)"),
  (Intent.CarIsWhat, r"(.*)是什么"),
]

In [6]:
def retrieve_kg(graph, db_query, uri2resource):
  print('SPARQL:')
  print(db_query)
  for row in graph.query(db_query):
    print('Output:', uri2resource[row.o.toPython()])
    print()
    print()

for utt in ['宝马是什么',
            '我想了解一下宝马',
            '给我介绍一下宝马',
            '给我讲解一下捷豹这个汽车品牌',
            '给我介绍一下帕加尼',
            '我想了解一下广汽本田',
            '宝马这个牌子的汽车怎么样',
            '宾利这个牌子的汽车怎么样',
            '北京奔驰怎么样',
            '宝马如何呢',
            '宝马汽车好用吗',
            '宝马和奔驰比怎么样',
            '宝马和奔驰比哪个好',
            '宝马和奔驰比哪个更好',]:
  print(utt)
  for intent, rule in rules:
    slots = re.findall(rule, utt)
    if len(slots) > 0:
      print('Intent:', intent)
      print('Rule:', rule)
      for slot in slots:
        if intent == Intent.CarIsWhat:
          db_query = (
          """
          PREFIX : <%s>

          SELECT DISTINCT ?o WHERE {
              :%s :is_what ?o .
          }
          """ % (URI_PREFIX, slot)
          )
          retrieve_kg(graph, db_query, uri2resource)
        if intent == Intent.CarIsHow:
          db_query = (
          """
          PREFIX : <%s>
          
          SELECT DISTINCT ?o WHERE {
              :%s :is_how ?o .
          }
          """ % (URI_PREFIX, slot)
          )
          retrieve_kg(graph, db_query, uri2resource)
        if intent == Intent.CarIsCompared:
          db_query = (
          """
          PREFIX : <%s>
          
          SELECT DISTINCT ?o WHERE {
              :%s :is_compared ?o .
          }
          """ % (URI_PREFIX, slot)
          )
          retrieve_kg(graph, db_query, uri2resource)
      break

宝马是什么
Intent: car_is_what
Rule: (.*)是什么
SPARQL:

          PREFIX : <http://kgdemo.com/>

          SELECT DISTINCT ?o WHERE {
              :宝马 :is_what ?o .
          }
          
Output: 宝马（BMW）是德国豪华汽车品牌


我想了解一下宝马
Intent: car_is_what
Rule: (?:我想|给我)(?:了解|介绍|讲解)(?:一下)?(.*)$
SPARQL:

          PREFIX : <http://kgdemo.com/>

          SELECT DISTINCT ?o WHERE {
              :宝马 :is_what ?o .
          }
          
Output: 宝马（BMW）是德国豪华汽车品牌


给我介绍一下宝马
Intent: car_is_what
Rule: (?:我想|给我)(?:了解|介绍|讲解)(?:一下)?(.*)$
SPARQL:

          PREFIX : <http://kgdemo.com/>

          SELECT DISTINCT ?o WHERE {
              :宝马 :is_what ?o .
          }
          
Output: 宝马（BMW）是德国豪华汽车品牌


给我讲解一下捷豹这个汽车品牌
Intent: car_is_what
Rule: (?:我想|给我)(?:了解|介绍|讲解)(?:一下)?(.*)(?:这个汽车品牌)
SPARQL:

          PREFIX : <http://kgdemo.com/>

          SELECT DISTINCT ?o WHERE {
              :捷豹 :is_what ?o .
          }
          
Output: 捷豹（Jaguar）英国豪华汽车品牌，英国皇室御用品牌，1935年诞生


给我介绍一下帕加尼
Intent: car_is_what
Rule: (?:我想|给